# Lab 2 Data Reading and Processing

Author: Zackary Lee
Submission Date: 10/13/2025

Project details: Given three distinct files (CSV, JSON, and TXT) 

FINISH LATER

In [1]:
#import statements
import pandas as pd
import numpy as np
import glob
import csv
import json
import re

In [2]:
# grab all the csv, json, and text files
csv_files = glob.glob("*.csv")
json_files = glob.glob("*.json")
txt_files = glob.glob("*.txt")
                    
csv_files

['fortune500.csv', 'processed_data.csv']

In [3]:
json_files

['lines.json']

In [4]:
txt_files

['unstructureddata.txt']

In [5]:
# create the file paths for each of the files
csv_file_path = 'fortune500.csv'
json_file_path = 'lines.json'
txt_file_path = 'unstructureddata.txt'

# 1.1 CSV Data Preprocessing

In [6]:
#read line by line, and filter the bad data
a = 0
csv_title = []
csv_gooddata = []
csv_baddata= []

# Iterate over the CSV file and seperate the good data from the bad data 
# If the row length matches the desired length append that row to the gooddata
# if the the row length does not match append the row to baddata
with open(csv_file_path, mode = 'r', encoding = 'utf-8') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        if a == 0:
            title = row
        a += 1
        if len(row) == 5 and a != 1:
            csv_gooddata.append(row)
        else:
            csv_baddata.append(row)
        

In [7]:
title

['#Year',
 '(1)Rank',
 '!Company',
 '(3)Revenue (in millions)',
 'okjb)Profit (in millions)']

In [8]:
#Clean up title
# 1) strip BOM/edges
title = [t.lstrip('\ufeff').strip() for t in title]

# 2) if there’s any ')' before the real name, drop everything up to and including that ')'
title = [re.sub(r'^[^)]*\)', '', t) for t in title]

# 3) now remove any leftover leading parentheses/spaces (this fixes the '()Rank' case)
title = [re.sub(r'^[()\s]+', '', t) for t in title]

# 4) clean the rest and normalize spacing
title = [re.sub(r'[^A-Za-z()\s]+', '', t) for t in title]
title = [re.sub(r'\s+', ' ', t).strip() for t in title]

title

['Year', 'Rank', 'Company', 'Revenue (in millions)', 'Profit (in millions)']

In [10]:
csv_gooddata

[['1955', '1', 'General Motors', '9823.5', '806'],
 ['1955', '2', 'Exxon Mobil', '5661.4', '584.8'],
 ['1955', '3', 'U.S. Steel', '3250.4', '195.4'],
 ['1955', '4', 'General Electric', '', '212.6'],
 ['1955', '5', 'Esmark', '2510.8', '19.1'],
 ['1955', '6', 'Chrysler', '2071.6', '18.5'],
 ['1955', '7', 'Armour', '2056.1', '1.6'],
 ['1955', '8', 'Gulf Oil', '1705.3', '182.8'],
 ['1955', '9', 'Mobil', '1703.6', '183.8'],
 ['1955', '10', 'DuPont', '1687.7', '344.4'],
 ['1955', '11', 'Amoco', '1667.4', '132.8'],
 ['1955', '12', 'Bethlehem Steel', '1660.3', '117.2'],
 ['1955', '13', 'CBS', '1631', '84.6'],
 ['1955', '14', 'Texaco', '1574.4', '226.1'],
 ['1955', '15', 'AT&T Technologies', '1526.2', '55.8'],
 ['1955', '16', 'Shell Oil', '1312.1', '121.1'],
 ['1955', '17', 'Kraft', '1210.3', '37.4'],
 ['1955', '18', 'ChevronTexaco', '1113.3', '211.9'],
 ['1955', '19', 'Goodyear Tire & Rubber', '1090.1', '48.1'],
 ['1955', '20', 'Boeing', '1033.2', '37'],
 ['1955', '21', 'Sinclair Oil', '1021.5

In [12]:
csv_baddata

[['#Year',
  '(1)Rank',
  '!Company',
  '(3)Revenue (in millions)',
  'okjb)Profit (in millions)'],
 ['1', '', '', 'l', '776.8', '22.7'],
 ['1955', '1'],
 ['1955', '', '', '', 'Du Mont (Allen B.) Labs.', '91.9', '0.9'],
 ['1955', '335', 'Eagle-Picher Industries', '', '', ''],
 ['', '', '', 'Rockwell Manufacturing', '76.5', '5.7'],
 ['1955', '412'],
 ['1956', '104', 'Crane', ''],
 ['19', '', '', ''],
 ['195', '', '', '6.9'],
 ['197', '', '', '.4'],
 ['1980', '186', 'Kopp2.4'],
 ['1980', '', '', ''],
 ['1980', '3', '', '', '1', '26.5'],
 ['1981', '57', 'Gulf & Western Industries', '', '', '']]

In [14]:
# put the gooddata into a Pandas DataFrame 
csv_df = pd.DataFrame(csv_gooddata, columns = title)

csv_df

,Year,Rank,Company,Revenue (in millions),Profit (in millions)
0,1955,1,General Motors,9823.5,806
1,1955,2,Exxon Mobil,5661.4,584.8
2,1955,3,U.S. Steel,3250.4,195.4
3,1955,4,General Electric,,212.6
4,1955,5,Esmark,2510.8,19.1
...,...,...,...,...,...
28838,2009,996,Tellabs,"1,729.00",-930.1
28839,2009,997,Administaff,"1,724.40",45.8
28840,2009,998,Sanderson Farms,"1,723.60",-43.1
28841,2009,999,MGIC Investment,"1,721.50",-518.9


# 1.2 JSON Data Preprocessing

In [15]:
# create an array for the json_data and for the title for each column
json_data = []
json_title = ['Year', 'Rank', 'Company', 'Revenue (in millions)', 'Profit (in millions)']

with open(json_file_path, 'r') as file:
    for line in file:
        try:
            # Parse the JSON string
            json_obj = json.loads(line)

            # Handle missing data by setting a default value
            for key in title:
                if key not in json_obj:
                        json_obj[key] = None

            # Append the processed JSON object to the list
            json_data.append(json_obj)
        except json.JSONDecodeError:
            print(f'Invalid Json data: {line}')



Invalid Json data: {"Year":1977,"Rank":,"Company":"Shell Oil","Revenue (in millions)":"9230","Profit (in millions)":"705.8"}

Invalid Json data: {"Year":1977,"Rank":17,"Company":"Conoco","Revenue (in millions)":,"Profit (in millions)":"460"}

Invalid Json data: {"Year":1977?,"Rank":20,"Company":"Tenneco Automotive","Revenue (in millions)":"6389.2","Profit (in millions)":"383.5"}

Invalid Json data: {"Year":1977,"Rank":21,"Company":,"Revenue (in millions)":"6345.7","Profit (in millions)":"441.2"}

Invalid Json data: {"Year":1977,"Rank":26,"Company":"Occidental Petroleum","Revenue (in millions)":,"Profit (in millions)":"183.7"}

Invalid Json data: {"Year":1977,"Rank":43,"Company":,"Revenue (in millions)":"4086.8","Profit (in millions)":"136"}

Invalid Json data: {"Year":1977aka,"Rank":381,"Company":"National Service Industries","Revenue (in millions)":"482.7","Profit (in millions)":"24.9"}

Invalid Json data: {"Year":1977,"Rank":418,"Company":"Kellwood","Revenue (in millions)":,"Profit (

In [16]:
# Create a pandas dataframe with the good data
json_df = pd.DataFrame(json_data, columns = json_title)

json_df

,Year,Rank,Company,Revenue (in millions),Profit (in millions)
0,1977,1,Exxon Mobil,48630.8,2641
1,1977,2,General Motors,47181,2902.8
2,1977,3,Ford Motor,28839.6,983.1
3,1977,4,Texaco,26451.9,869.7
4,1977,5,Mobil,26062.6,942.5
...,...,...,...,...,...
10869,1998,396,American Family Ins. Grp.,3689.4,251.6
10870,1998,397,Baker Hughes,3685.4,97
10871,1998,398,Service Merchandise,3662.8,-91.6
10872,1998,399,Silicon Graphics,3662.6,78.6


# 1.3 TXT Data Preprocessing

In [17]:
#Function to process a block of text and return a dictionary
def process_block(block):
    entity={}
    for item in block:
        key, value = item.split(':')
        entity[key] = value
    return entity

# Read and process the file
current_data = []
result = []
with open(txt_file_path,'r') as file:
    for line in file:
        if line.strip() == "":

            dfcontent = process_block(current_data)
            result.append(dfcontent)
            current_data = []
        else: 
            current_data.append(line.strip())

In [18]:
# Save the txt data to a pandas DataFrame
txt_df = pd.DataFrame(result)

txt_df

,Year,Rank,Company,Revenue (in millions),Profit (in millions)
0,1998,401,Turner Corp.,3639.8,5.9
1,1998,402,Reebok International,3637.4,135.1
2,1998,403,Morton International,3636.5,343
3,1998,404,Engelhard,3630.7,47.8
4,1998,405,Temple-Inland,3625.4,50.8
...,...,...,...,...,...
7595,2009,996,Tellabs,"1,729.00",-930.1
7596,2009,997,Administaff,"1,724.40",45.8
7597,2009,998,Sanderson Farms,"1,723.60",-43.1
7598,2009,999,MGIC Investment,"1,721.50",-518.9


# 2.1 Combine the three Pandas DataFrames and Standarize Column Names

In [19]:
def normalize_cols(cols):
    new = {}
    for c in cols:
        k = re.sub(r'\s+', ' ', re.sub(r'[^A-Za-z0-9 ]', ' ', str(c))).strip().lower()
        if k.startswith('year'):
            new[c] = 'Year'
        elif k.startswith('rank'):
            new[c] = 'Rank'
        elif k.startswith('company'):
            new[c] = 'Company'
        elif k.startswith('revenue'):
            new[c] = 'Revenue'
        elif k.startswith('profit'):
            new[c] = 'Profit'
    return new

def normalize_frame(df):
    df = df.rename(columns=normalize_cols(df.columns))
    # Keep only needed columns
    return df[['Year', 'Rank', 'Company', 'Revenue', 'Profit']]

# Apply normalization BEFORE merge
csv_df  = normalize_frame(csv_df)
json_df = normalize_frame(json_df)
txt_df  = normalize_frame(txt_df)

#Now create result_df safely
result_df = pd.concat([csv_df, json_df, txt_df], ignore_index=True, sort=False)
result_df


,Year,Rank,Company,Revenue,Profit
0,1955,1,General Motors,9823.5,806
1,1955,2,Exxon Mobil,5661.4,584.8
2,1955,3,U.S. Steel,3250.4,195.4
3,1955,4,General Electric,,212.6
4,1955,5,Esmark,2510.8,19.1
...,...,...,...,...,...
47312,2009,996,Tellabs,"1,729.00",-930.1
47313,2009,997,Administaff,"1,724.40",45.8
47314,2009,998,Sanderson Farms,"1,723.60",-43.1
47315,2009,999,MGIC Investment,"1,721.50",-518.9


In [20]:
# Clean and extract valid numeric Year values
year_extracted = result_df['Year'].astype(str).str.extract(r'(\d{4})', expand=False)
result_df['Year'] = pd.to_numeric(year_extracted, errors='coerce')

# Identify invalid year rows (optional)
bad_year_rows = result_df[result_df['Year'].isna()]

#Create a separate filtered dataframe for 1995–1998
result_df_95_98 = result_df[result_df['Year'].between(1995, 1998, inclusive='both')].copy()

# Convert Year to integer for neatness
result_df_95_98['Year'] = result_df_95_98['Year'].astype('Int64')

# Show the filtered result
result_df_95_98



,Year,Rank,Company,Revenue,Profit
19776,1995,1,General Motors,154951.2,4900.6
19777,1995,2,Ford Motor,128439,5308
19778,1995,3,Exxon Mobil,101459,5100
19779,1995,4,Wal-Mart Stores,83412.4,2681
19780,1995,5,AT&T,75094,4676
...,...,...,...,...,...
39812,1998,496,Barnes & Noble,2796.9,53.2
39813,1998,497,Pacific Enterprises,2777,184
39814,1998,498,Xcel Energy,2733.7,237.3
39815,1998,499,Maxxam,2729.1,65.2


# 2.2 Find the Highest Revenue and Profit

In [21]:
# Find company with highest revenue from 1995 to 1998
max_revenue_row = result_df.loc[result_df['Revenue'].idxmax()]
highest_revenue_company = max_revenue_row['Company']
highest_revenue_value = max_revenue_row['Revenue']
highest_revenue_year = int(max_revenue_row['Year'])

# Find company with highest profit from 1995 to 1998
max_profit_row = result_df.loc[result_df['Profit'].idxmax()]
highest_profit_company = max_profit_row['Company']
highest_profit_value = max_profit_row['Profit']
highest_profit_year = int(max_profit_row['Year'])

print("Highest Revenue Company (1995–1998):", highest_revenue_company, "-", highest_revenue_value, "in", highest_revenue_year)
print("Highest Profit Company (1995–1998):", highest_profit_company, "-", highest_profit_value, "in", highest_profit_year)


Highest Revenue Company (1995–1998): Kelsey-Hayes - wregwg in 1955
Highest Profit Company (1995–1998): Ex-Cell-O - fs in 1955


# 2.3 Find the Unique Companies, Good Data, and Bad Data

In [22]:

# --- 1) GOOD and BAD ---
good_data_count = len(result_df)  # good = everything in the final cleaned dataset

# total rows from all files before cleaning
total_original_rows = len(csv_df) + len(json_df) + len(txt_df)
bad_data_count = total_original_rows - good_data_count

# --- 2) Unique companies (ALL YEARS) ---
unique_companies_count = result_df['Company'].dropna().nunique()

# --- 3) Winners in 1995–1998 (make sure numeric & non-NaN) ---
# Ensure numeric (idempotent if already cleaned)
result_df_95_98 = result_df_95_98.copy()
result_df_95_98['Revenue'] = pd.to_numeric(result_df_95_98['Revenue'], errors='coerce')
result_df_95_98['Profit']  = pd.to_numeric(result_df_95_98['Profit'],  errors='coerce')

winners = result_df_95_98.dropna(subset=['Revenue', 'Profit'])
if winners.empty:
    raise ValueError("No rows in 1995–1998 with both Revenue and Profit present after cleaning.")

max_revenue_row = winners.loc[winners['Revenue'].idxmax()]
max_profit_row  = winners.loc[winners['Profit'].idxmax()]

def fmt_money(x):
    return f"${float(x):,.0f}"  # no decimals, add thousands separators

highest_revenue_company = f"{max_revenue_row['Company']} ({int(max_revenue_row['Year'])}) - {fmt_money(max_revenue_row['Revenue'])}"
highest_profit_company  = f"{max_profit_row['Company']} ({int(max_profit_row['Year'])}) - {fmt_money(max_profit_row['Profit'])}"

# --- 4) Build Results_Combine ---
results_combine = pd.DataFrame({
    "Metric": [
        "Total Good Data Rows",
        "Total Bad Data Rows",
        "Unique Companies",
        "Highest Revenue Company (1995–1998)",
        "Highest Profit Company (1995–1998)"
    ],
    "Value": [
        good_data_count,
        bad_data_count,
        unique_companies_count,
        highest_revenue_company,
        highest_profit_company
    ]
})

results_combine


,Metric,Value
0,Total Good Data Rows,47317
1,Total Bad Data Rows,0
2,Unique Companies,4137
3,Highest Revenue Company (1995–1998),"General Motors (1998) - $178,174"
4,Highest Profit Company (1995–1998),"Exxon Mobil (1998) - $8,460"


# 3.1 Save the Final Result to CSV File

In [23]:
results_combine.to_csv("processed_data.csv", index = False)